<a href="https://colab.research.google.com/github/Deltan2002/pytorch/blob/main/MNIST_neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)


  def forward(self, x):
    x= F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


model = NN(784, 10)
x = torch.rand(64,784)
print(model(x).shape)

torch.Size([64, 10])


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [9]:
input_size = 784
num_classes = 10
lr = 0.001
batch_size = 64
epochs = 10

In [10]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [11]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [12]:
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [ ]:
for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda
        data = data.to(device=device)
        targets = targets.to(device=device)

        # reshaping
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criteria(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

        if batch_idx % 200 == 0:
            print(f"Epoch {epoch}/{epochs}, Batch {batch_idx}, Loss: {loss.item()}")


In [16]:
def check_accuracy(loader, model):

  num_correct, num_samples = 0,0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)


      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"Got {num_correct}/{num_samples} with accuracy {(float(num_correct) / float(num_samples)) * 100:.2f}%")


  model.train()


In [17]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 59302/60000 with accuracy 98.84%
Got 9709/10000 with accuracy 97.09%
